Code to parse Azure cloud service data

This implementation includes several key features:
Web Scraping: Uses requests and BeautifulSoup4 to fetch and parse the Azure VM availability documentation.
Content Processing: Removes unnecessary HTML elements, Organizes content into sections based on headers, Maintains the structure of the documentation
Question Answering: Uses BERT model fine-tuned on SQuAD (Stanford Question Answering Dataset), Finds the most relevant section for each question, Generates answers based on the context
Interactive Interface: Provides a simple command-line chat interface, Allows users to ask questions until they choose to quit
To use the chatbot, simply run the script and start asking questions about Azure VM availability. For example:

# Example usage
chatbot = AzureVMDocChatbot()

# Ask questions
print(chatbot.answer_question("What are availability zones in Azure?"))
print(chatbot.answer_question("How does Azure Site Recovery help with business continuity?"))

The chatbot will provide answers based on the documentation content. The answers are generated using the BERT model, which understands context and can provide relevant responses based on the documentation.
Some example questions you can ask:
What are availability zones?
How does Azure ensure storage redundancy?
What is an availability set?
How does Azure Site Recovery work?
What is a Virtual Machine Scale Set?
Note that the quality of answers depends on:
The relevance of the section matching
The quality of the question
The presence of the information in the documentation
You can enhance this implementation by:
Adding better error handling
Implementing more sophisticated section matching
Adding support for follow-up questions
Implementing a web interface instead of command-line
Adding support for multiple documentation sources
Would you like me to explain any particular part of the implementation in more detail?

Import required libraries

In [5]:
import requests
from bs4 import BeautifulSoup
import re
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import logging
from urllib.parse import urljoin, urlparse
from collections import defaultdict
import json
import pickle
import os
from sentence_transformers import SentenceTransformer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Next steps

In [6]:


class AzureVMDocChatbot:
    def __init__(self, timeout=30, max_depth=2, max_pages=10, data_dir='cached_data'):
        # Set up logging first
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)
        
        # Initialize attributes
        self.base_url = "https://learn.microsoft.com/en-us/azure/virtual-machines/"
        self.doc_content = ""
        self.sections = {}
        self.timeout = timeout
        self.max_depth = max_depth
        self.max_pages = max_pages
        self.visited_urls = set()
        self.url_to_sections = defaultdict(dict)
        self.data_dir = data_dir
        self.embeddings = {}
        
        # Create data directory if it doesn't exist
        os.makedirs(data_dir, exist_ok=True)
        
        # Initialize models
        self.init_models()
        
        # Try to load cached data first, if not available then crawl
        if not self.load_cached_data():
            self.logger.info("No cached data found. Starting fresh crawl...")
            self.crawl_and_parse_content()
            self.generate_embeddings()
            self.save_cached_data()

    def init_models(self):
        """Initialize the BERT and sentence transformer models"""
        self.logger.info("Loading models...")
        start_time = time.time()
        try:
            # QA model
            self.tokenizer = AutoTokenizer.from_pretrained(
                "bert-large-uncased-whole-word-masking-finetuned-squad",
                local_files_only=False
            )
            self.qa_model = AutoModelForQuestionAnswering.from_pretrained(
                "bert-large-uncased-whole-word-masking-finetuned-squad",
                local_files_only=False
            )
            
            # Sentence embedding model
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            
            self.logger.info(f"Models loaded successfully in {time.time() - start_time:.2f} seconds!")
        except Exception as e:
            self.logger.error(f"Error loading models: {str(e)}")
            raise

    def save_cached_data(self):
        """Save parsed content and embeddings to disk"""
        self.logger.info("Saving cached data...")
        try:
            # Save sections and URL mapping
            with open(os.path.join(self.data_dir, 'sections.json'), 'w', encoding='utf-8') as f:
                json.dump(self.sections, f, ensure_ascii=False, indent=2)
            
            with open(os.path.join(self.data_dir, 'url_to_sections.json'), 'w', encoding='utf-8') as f:
                json.dump(dict(self.url_to_sections), f, ensure_ascii=False, indent=2)
            
            # Save embeddings using pickle
            with open(os.path.join(self.data_dir, 'embeddings.pkl'), 'wb') as f:
                pickle.dump(self.embeddings, f)
                
            self.logger.info("Cache saved successfully!")
        except Exception as e:
            self.logger.error(f"Error saving cache: {str(e)}")

    def load_cached_data(self):
        """Load cached content and embeddings if available"""
        try:
            # Check if all required cache files exist
            cache_files = ['sections.json', 'url_to_sections.json', 'embeddings.pkl']
            if not all(os.path.exists(os.path.join(self.data_dir, f)) for f in cache_files):
                return False
                
            self.logger.info("Loading cached data...")
            
            # Load sections and URL mapping
            with open(os.path.join(self.data_dir, 'sections.json'), 'r', encoding='utf-8') as f:
                self.sections = json.load(f)
            
            with open(os.path.join(self.data_dir, 'url_to_sections.json'), 'r', encoding='utf-8') as f:
                self.url_to_sections = defaultdict(dict, json.load(f))
            
            # Load embeddings
            with open(os.path.join(self.data_dir, 'embeddings.pkl'), 'rb') as f:
                self.embeddings = pickle.load(f)
                
            self.logger.info("Cache loaded successfully!")
            return True
        except Exception as e:
            self.logger.error(f"Error loading cache: {str(e)}")
            return False

    def is_valid_azure_doc_url(self, url):
        """Check if URL is a valid Azure VM documentation page"""
        parsed = urlparse(url)
        return (
            parsed.netloc == "learn.microsoft.com" and
            "/azure/virtual-machines" in parsed.path and
            not any(ext in url for ext in ['.png', '.jpg', '.gif', '.pdf'])
        )

    def extract_links(self, soup, current_url):
        """Extract valid Azure documentation links from the page"""
        links = set()
        for a in soup.find_all('a', href=True):
            href = a['href']
            absolute_url = urljoin(current_url, href)
            if self.is_valid_azure_doc_url(absolute_url) and absolute_url not in self.visited_urls:
                links.add(absolute_url)
        return links

    def parse_page_content(self, soup, url):
        """Parse content from a single page"""
        main_content = soup.find('main')
        if not main_content:
            return
        
        # Remove unnecessary elements
        for element in main_content.find_all(['script', 'style', 'nav']):
            element.decompose()
        
        # Parse sections
        headers = main_content.find_all(['h1', 'h2', 'h3'])
        current_section = ""
        current_content = []
        
        for header in headers:
            if current_section:
                self.url_to_sections[url][current_section] = ' '.join(current_content)
            current_section = header.get_text(strip=True)
            current_content = []
            
            next_element = header.find_next_sibling()
            while next_element and not next_element.name in ['h1', 'h2', 'h3']:
                if next_element.get_text(strip=True):
                    current_content.append(next_element.get_text(strip=True))
                next_element = next_element.find_next_sibling()
        
        # Add the last section
        if current_section:
            self.url_to_sections[url][current_section] = ' '.join(current_content)

    def crawl_and_parse_content(self):
        """Crawl through pages and parse content with depth limit"""
        try:
            to_visit = [(self.base_url, 0)]  # (url, depth)
            pages_processed = 0
            
            self.logger.info("Starting content crawling and parsing...")
            start_time = time.time()
            
            with tqdm(total=self.max_pages, desc="Processing pages") as pbar:
                while to_visit and pages_processed < self.max_pages:
                    current_url, depth = to_visit.pop(0)
                    
                    if current_url in self.visited_urls:
                        continue
                    
                    self.logger.info(f"\nProcessing: {current_url}")
                    try:
                        response = requests.get(current_url, timeout=self.timeout)
                        response.raise_for_status()
                        
                        soup = BeautifulSoup(response.text, 'html.parser')
                        self.parse_page_content(soup, current_url)
                        self.visited_urls.add(current_url)
                        pages_processed += 1
                        pbar.update(1)
                        
                        # If we haven't reached max depth, add child links to visit
                        if depth < self.max_depth:
                            new_links = self.extract_links(soup, current_url)
                            to_visit.extend((link, depth + 1) for link in new_links)
                            
                    except requests.RequestException as e:
                        self.logger.error(f"Error fetching {current_url}: {str(e)}")
                        continue
            
            total_time = time.time() - start_time
            self.logger.info(f"\nCrawling completed in {total_time:.2f} seconds")
            self.logger.info(f"Processed {pages_processed} pages")
            
            # Combine all sections for searching
            for url, sections in self.url_to_sections.items():
                self.sections.update({f"{k} ({url})": v for k, v in sections.items()})
                
        except Exception as e:
            self.logger.error(f"Error during crawling: {str(e)}")
            raise

    def generate_embeddings(self):
        """Generate embeddings for all sections"""
        self.logger.info("Generating embeddings for sections...")
        try:
            for section_key, content in tqdm(self.sections.items(), desc="Generating embeddings"):
                # Generate embedding for the content
                embedding = self.embedding_model.encode(content)
                self.embeddings[section_key] = embedding
            
            self.logger.info(f"Generated embeddings for {len(self.embeddings)} sections")
        except Exception as e:
            self.logger.error(f"Error generating embeddings: {str(e)}")
            raise

    def find_most_relevant_section(self, question):
        """Find the most relevant section using embeddings"""
        self.logger.info("Searching for relevant section...")
        start_time = time.time()
        
        try:
            # Generate embedding for the question
            question_embedding = self.embedding_model.encode(question)
            
            # Find most similar section using cosine similarity
            max_similarity = -1
            best_section = None
            best_url = None
            
            for section_key, section_embedding in self.embeddings.items():
                similarity = np.dot(question_embedding, section_embedding) / (
                    np.linalg.norm(question_embedding) * np.linalg.norm(section_embedding)
                )
                
                if similarity > max_similarity:
                    max_similarity = similarity
                    content = self.sections[section_key]
                    best_section = content
                    best_url = section_key[section_key.rfind("(")+1:-1] if "(" in section_key else None
            
            search_time = time.time() - start_time
            self.logger.info(f"Found relevant section in {search_time:.2f} seconds")
            if best_url:
                self.logger.info(f"Source: {best_url}")
            return best_section
            
        except Exception as e:
            self.logger.error(f"Error finding relevant section: {str(e)}")
            return None

    def answer_question(self, question):
        """Answer a question using the crawled content"""
        self.logger.info(f"\nProcessing question: {question}")
        start_time = time.time()
        
        context = self.find_most_relevant_section(question)
        if not context:
            return "I'm sorry, I couldn't find relevant information to answer your question."

        self.logger.info("Generating answer...")
        inputs = self.tokenizer(question, context, return_tensors="pt", max_length=512, truncation=True)
        
        with torch.no_grad():
            outputs = self.qa_model(**inputs)
        
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits)
        
        tokens = self.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        answer = tokens[answer_start:answer_end + 1]
        answer = self.tokenizer.convert_tokens_to_string(answer)
        
        total_time = time.time() - start_time
        self.logger.info(f"Answer generated in {total_time:.2f} seconds")
        
        return answer if answer else "I'm sorry, I couldn't generate a good answer for that question."

    def demonstrate_embeddings(self):
        """Demonstrate how embeddings work with simple examples"""
        self.logger.info("Demonstrating embeddings...")
        
        # Example texts
        texts = [
            "Azure Virtual Machines",
            "Azure VMs",
            "Creating a VM in Azure",
            "Kubernetes cluster",  # Different concept
        ]
        
        # Generate embeddings
        embeddings = self.embedding_model.encode(texts)
        
        # Compare similarities
        self.logger.info("\nSimilarity scores:")
        for i, text1 in enumerate(texts):
            for j, text2 in enumerate(texts[i+1:], i+1):
                similarity = np.dot(embeddings[i], embeddings[j]) / (
                    np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j])
                )
                self.logger.info(f"'{text1}' vs '{text2}': {similarity:.3f}")

def test_chatbot():
    print("Initializing chatbot...")
    chatbot = AzureVMDocChatbot(timeout=30, max_depth=2, max_pages=10)
    
    # Test some questions
    test_questions = [
        "What are Azure Virtual Machines?",
        "How do I create a VM in Azure?",
        "What are the latest features in Azure VMs?",
        "What is Azure Boost?",
        "How does VM hibernation work?"
    ]
    
    for question in test_questions:
        print(f"\nQ: {question}")
        answer = chatbot.answer_question(question)
        print(f"A: {answer}")
        print("-" * 80)

if __name__ == "__main__":
    test_chatbot()

INFO:__main__:Loading models...


Initializing chatbot...


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Models loaded successfully in 8.26 seconds!
INFO:__main__:No cached data found. Starting fresh crawl...
INFO:__main__:S


Q: What are Azure Virtual Machines?


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.62it/s]
INFO:__main__:Found relevant section in 0.08 seconds
INFO:__main__:Source: https://learn.microsoft.com/en-us/azure/virtual-machines/
INFO:__main__:Generating answer...
INFO:__main__:Answer generated in 4.73 seconds
INFO:__main__:
Processing question: How do I create a VM in Azure?
INFO:__main__:Searching for relevant section...


...

A: documentation
--------------------------------------------------------------------------------

Q: What is Azure Boost?


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.89it/s]
INFO:__main__:Found relevant section in 0.06 seconds
INFO:__main__:Source: https://learn.microsoft.com/en-us/azure/virtual-machines/
INFO:__main__:Generating answer...
INFO:__main__:Answer generated in 0.33 seconds
INFO:__main__:
Processing question: How does VM hibernation work?
INFO:__main__:Searching for relevant section...


A: virtual machine scale sets
--------------------------------------------------------------------------------

Q: How does VM hibernation work?


Batches: 100%|██████████| 1/1 [00:00<00:00, 23.49it/s]
INFO:__main__:Found relevant section in 0.05 seconds
INFO:__main__:Source: https://learn.microsoft.com/en-us/azure/virtual-machines/hibernate-resume
INFO:__main__:Generating answer...
INFO:__main__:Answer generated in 0.53 seconds


A: check out thewindows hibernation documentation
--------------------------------------------------------------------------------


In [7]:
# Cell 3: Initialize the chatbot
chatbot = AzureVMDocChatbot()

INFO:__main__:Loading models...
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:Models loaded successfully in 3.21 seconds!
INFO:__main__:Loading cached data...
INFO:_

In [8]:
# Cell 4: Visualize the parsed sections
sections_df = pd.DataFrame(list(chatbot.sections.items()), columns=['Section', 'Content'])
print(f"Total number of sections: {len(sections_df)}")
sections_df.head()

Total number of sections: 91


,Section,Content
0,Virtual machines in Azure (https://learn.micro...,Documentation for creating and managing virtua...
1,Latest features (https://learn.microsoft.com/e...,
2,What's new (https://learn.microsoft.com/en-us/...,Azure BoostHibernationFlexible Virtual Machine...
3,Linux quickstarts (https://learn.microsoft.com...,
4,Quickstart (https://learn.microsoft.com/en-us/...,Azure portalAzure PowerShellTerraformAzure CLI


In [9]:
# Cell 5: Test some example questions
test_questions = [
    "What are availability zones?",
    "How does Azure ensure storage redundancy?",
    "What is an availability set?",
    "How does Azure Site Recovery work?",
    "What is a Virtual Machine Scale Set?",
    "What is the difference between availability zones and availability sets?",
    "How many availability zones are there in an Azure region?",
    "How does Azure Site Recovery help with business continuity?",  
    "How does Azure Load Balancer work with availability zones?"
]

for question in test_questions:
    print(f"\nQ: {question}")
    print(f"A: {chatbot.answer_question(question)}")
    print("-" * 80)

INFO:__main__:
Processing question: What are availability zones?
INFO:__main__:Searching for relevant section...



Q: What are availability zones?


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.38it/s]
INFO:__main__:Found relevant section in 0.13 seconds
INFO:__main__:Source: https://learn.microsoft.com/en-us/azure/virtual-machines/flexible-virtual-machine-scale-sets
INFO:__main__:Generating answer...
INFO:__main__:Answer generated in 2.91 seconds
INFO:__main__:
Processing question: How does Azure ensure storage redundancy?
INFO:__main__:Searching for relevant section...


A: fault domains
--------------------------------------------------------------------------------

Q: How does Azure ensure storage redundancy?


Batches: 100%|██████████| 1/1 [00:00<00:00, 30.58it/s]
INFO:__main__:Found relevant section in 0.04 seconds
INFO:__main__:Source: https://learn.microsoft.com/en-us/azure/virtual-machines/
INFO:__main__:Generating answer...
INFO:__main__:Answer generated in 0.45 seconds
INFO:__main__:
Processing question: What is an availability set?
INFO:__main__:Searching for relevant section...


...

A: I'm sorry, I couldn't generate a good answer for that question.
--------------------------------------------------------------------------------

Q: How does Azure Site Recovery help with business continuity?


Batches: 100%|██████████| 1/1 [00:00<00:00, 33.00it/s]
INFO:__main__:Found relevant section in 0.04 seconds
INFO:__main__:Source: https://learn.microsoft.com/en-us/azure/virtual-machines/flexible-virtual-machine-scale-sets
INFO:__main__:Generating answer...
INFO:__main__:Answer generated in 1.75 seconds
INFO:__main__:
Processing question: How does Azure Load Balancer work with availability zones?
INFO:__main__:Searching for relevant section...


A: [CLS]
--------------------------------------------------------------------------------

Q: How does Azure Load Balancer work with availability zones?


Batches: 100%|██████████| 1/1 [00:00<00:00, 35.92it/s]
INFO:__main__:Found relevant section in 0.04 seconds
INFO:__main__:Source: https://learn.microsoft.com/en-us/azure/virtual-machines/flexible-virtual-machine-scale-sets
INFO:__main__:Generating answer...
INFO:__main__:Answer generated in 1.75 seconds


A: distributing vms
--------------------------------------------------------------------------------


In [ ]:
# Cell 6: Interactive question answering
from IPython.display import clear_output

def interactive_qa():
    while True:
        question = input("Ask a question (or type 'quit' to exit): ")
        if question.lower() == 'quit':
            break
            
        clear_output(wait=True)
        print(f"Q: {question}")
        print(f"A: {chatbot.answer_question(question)}")
        print("\n" + "-" * 80 + "\n")

interactive_qa()

INFO:__main__:
Processing question: What is Azure availability zone?
INFO:__main__:Searching for relevant section...
INFO:__main__:Found relevant section in 0.01 seconds
INFO:__main__:Source: https://learn.microsoft.com/en-us/azure/virtual-machines/availability
INFO:__main__:Generating answer...


Q: What is Azure availability zone?


INFO:__main__:Answer generated in 1.09 seconds


A: a physically separate zone

--------------------------------------------------------------------------------

